In [43]:
from typing import Dict

import pydriller
import pathlib

In [44]:
test_modules_path = pathlib.Path("test_submodules")
production_modules_path = pathlib.Path("production_submodules")

In [45]:
from typing import Literal
from collections import defaultdict

files: Dict[str, int] = {}
counters: Dict[Literal["tdd", "same", "non-tdd"], int] = defaultdict(int)

In [46]:
def is_test_file(filename: str) -> bool:
    return filename.endswith("Test.java")

In [47]:
def get_code_file_name(filename: str) -> str:
    return filename.replace("Test.java", ".java")

In [48]:
def get_test_file_name(filename: str) -> str:
    return filename.replace(".java", "Test.java")

In [49]:
for index, commit in enumerate(pydriller.Repository(str((test_modules_path / "dubboj").absolute())).traverse_commits()):
    # loop from the first commit to the last 
    
    files_in_commit = {modified.filename: modified for modified in commit.modified_files}
    
    for modified_file in commit.modified_files:
        if is_test_file(modified_file.filename) and modified_file.filename not in files:
            code_file_name = get_code_file_name(modified_file.filename)
            
            if code_file_name in files_in_commit:
                # same commit 
                counters["same"] += 1
            else:
                if code_file_name in files:
                    # seen code before test
                    counters["non-tdd"] += 1
        elif not is_test_file(modified_file.filename):
            test_file_name = get_test_file_name(modified_file.filename)
            
            if test_file_name not in files_in_commit and test_file_name in files:
                # test before this code
                counters["tdd"] += 1
    
    for file_name in files_in_commit.keys():
        files[file_name] = index
        

defaultdict(<class 'int'>, {})
defaultdict(<class 'int'>, {'same': 50})
defaultdict(<class 'int'>, {'same': 50})
defaultdict(<class 'int'>, {'same': 50})
defaultdict(<class 'int'>, {'same': 50})
defaultdict(<class 'int'>, {'same': 50})
defaultdict(<class 'int'>, {'same': 50})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 1})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 1})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 1})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 2})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 2})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 3})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 4})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 4})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 5})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 5})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 6})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 7})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 7})
defaultdict(<class 'int'>, {'same': 50, 'tdd': 9})
defaultdi

KeyboardInterrupt: 

In [ ]:
# print(files)
print(counters)